# Notes

## Day and short-term strategies

- Scalping: Taking many very small intraday moves, aiming for a few cents or ticks per trade, often with large size and high frequency.
- Momentum trading: Buying strength (or shorting weakness) and “riding the wave” as price and volume accelerate in one direction.
- Range trading: Buying near support and selling near resistance when an asset is oscillating in a well-defined price range.
- Breakout trading: Entering when price pushes through a key support/resistance level or consolidation range, expecting a strong follow-through move.
- News-based trading: Reacting to catalysts like earnings, economic data, or company headlines that cause sudden volatility and volume spikes.[1][3][4][5][7]

## What EMA means in trading terms

- It's a smoothed line on the chart that represents the average price over the last N periods, with recent candles influencing it more than older ones.  
- A “20 EMA” on a 5-minute chart = the exponentially weighted average of the last 20 five-minute closes, updating each bar.  
- Traders use it to:
  - Identify trend direction (price above a rising EMA = uptrend bias, below a falling EMA = downtrend bias).  
  - Act as “dynamic support/resistance” for pullback entries (buy near a rising EMA, sell near a fallin

  So when you see code like `df["ema20"]`, that column is just “the 20-period exponential moving average of price,” used as a faster, more responsive trend line than an SMA.

## Volume Impact

Volume refers to the number of shares or contracts traded for a particular stock during a given period, such as a minute, hour, or day. In day trading, volume is a key indicator showing the strength or interest in a stock's price movement.

### Importance of Volume in Day Trading

- **Liquidity:** Higher volume usually means the stock is easier to buy and sell quickly, making it more suitable for day trading.
- **Confirmation:** Traders often look for high volume to confirm the strength of a price trend or pattern. For example, a breakout above resistance is considered more reliable if it's accompanied by increased volume.
- **Volatility:** Surges in volume can signal heightened interest and potentially greater price swings, which create more opportunities for quick profits—or losses.

### Popular Strategies Incorporating Volume

- Volume Breakouts: Traders buy when a stock breaks above a price resistance on high volume, expecting the momentum to continue.
- Volume Reversals: Sudden spikes in volume after a prolonged trend may signal a reversal or exhaustion, prompting traders to take the opposite position.
- Volume Oscillators: Some strategies use indicators like the Volume Weighted Average Price (VWAP) or On Balance Volume (OBV) to find buy or sell signals based on volume trends.

Volume provides insights into crowd behavior and market dynamics, making it one of the most watched metrics by day traders. By analyzing volume patterns, traders aim to improve timing and confidence in their trades.

## Relative Strength Index (RSI)

The RSI measures the velocity and magnitude of price movements. It ranges from 0 to 100 and is commonly used to identify overbought or oversold conditions.

In [1]:
"""
def calculate_rsi(delta:pd.Series, window:int=14, smooth:bool=False) -> float:
   # Delta is changes in price i.e. data.diff().
   global avg_gain, avg_loss
   rsi = 0
   gains = (delta.where(delta > 0, 0)).rolling(window=window).mean()
   losses = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

   # Initial 14 days, and the formula uses a positive value for the average loss.
   if not smooth:
      avg_gain = gains.rolling(window=window).mean()
      avg_loss = (-losses).rolling(window=window).mean()
      
   # Smooth the results so that the RSI only nears 100 or zero in a strongly trending market.
   else: 
      curr_gain = 0
      curr_loss = 0
      if delta[-1] > 0:
         curr_gain = delta[-1]
      else: 
         curr_loss = delta[-1]
      avg_gain = ((avg_gain * 13) + curr_gain) / window
      avg_loss = ((avg_loss * 13) + curr_loss) / window

   rs = avg_gain / avg_loss
   rsi = 100 - (100 / (1 + rs))

   return rsi
"""

'\ndef calculate_rsi(delta:pd.Series, window:int=14, smooth:bool=False) -> float:\n   # Delta is changes in price i.e. data.diff().\n   global avg_gain, avg_loss\n   rsi = 0\n   gains = (delta.where(delta > 0, 0)).rolling(window=window).mean()\n   losses = (-delta.where(delta < 0, 0)).rolling(window=window).mean()\n\n   # Initial 14 days, and the formula uses a positive value for the average loss.\n   if not smooth:\n      avg_gain = gains.rolling(window=window).mean()\n      avg_loss = (-losses).rolling(window=window).mean()\n\n   # Smooth the results so that the RSI only nears 100 or zero in a strongly trending market.\n   else: \n      curr_gain = 0\n      curr_loss = 0\n      if delta[-1] > 0:\n         curr_gain = delta[-1]\n      else: \n         curr_loss = delta[-1]\n      avg_gain = ((avg_gain * 13) + curr_gain) / window\n      avg_loss = ((avg_loss * 13) + curr_loss) / window\n\n   rs = avg_gain / avg_loss\n   rsi = 100 - (100 / (1 + rs))\n\n   return rsi\n'

## Moving Average Convergence Divergence (MACD)

The MACD is a trend-following momentum indicator that illustrates the relationship between two moving averages of a security’s price.

In [2]:
"""
The signal line is a nine-period EMA of the MACD line.
MACD is best used with daily periods, where the traditional settings of 26/12/9 days are the default.

def calculate_macd(data:pd.Series, short_window:int=12, long_window:int=26, signal_window:int=9):
   short_ema = data.ewm(span=short_window, adjust=False).mean()
   long_ema = data.ewm(span=long_window, adjust=False).mean()
   macd = short_ema - long_ema
   signal = macd.ewm(span=signal_window)
   return macd, signal
"""

'\nThe signal line is a nine-period EMA of the MACD line.\nMACD is best used with daily periods, where the traditional settings of 26/12/9 days are the default.\n\ndef calculate_macd(data:pd.Series, short_window:int=12, long_window:int=26, signal_window:int=9):\n   short_ema = data.ewm(span=short_window, adjust=False).mean()\n   long_ema = data.ewm(span=long_window, adjust=False).mean()\n   macd = short_ema - long_ema\n   signal = macd.ewm(span=signal_window)\n   return macd, signal\n'

## Walk-forward analysis vs back testing

A simulated live trading test on historical data (often called walk-forward analysis, market replay, or bar replay simulation) mimics real-time trading by revealing data gradually as if you're trading live, rather than seeing the full history upfront.​

How it works (vs regular backtest)

| Regular Backtest                          | Simulated Live Trading                                         |
| ----------------------------------------- | -------------------------------------------------------------- |
| Sees all data at once; knows future prices | Data revealed bar-by-bar; can't peek ahead                     |
| Tests full dataset immediately            | Replay mode: play/pause/rewind historical bars at chosen speed |
| Fast but can overfit                      | Slower but realisticdecision-making under time pressure        |

# Code

In [3]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import numpy as np
import time

In [4]:
# Globals
avg_gain = avg_loss = 0.0

In [5]:
# sofi = yf.Ticker("SOFI")

In [6]:
day = "2025-12-9"
start = pd.to_datetime(f"{day} 09:25")
end = pd.to_datetime(f"{day} 16:00")

tickers_df = yf.download("SOFI", start=start, end=end, interval="1m", auto_adjust=True).tz_convert("America/New_York")
tickers_df.dropna(inplace=True)

sofi_df = tickers_df.xs("SOFI", level="Ticker", axis=1)
sofi_df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Datetime,,,,,
2025-12-09 09:30:00-05:00,27.145000,27.610001,27.129999,27.610001,963090
2025-12-09 09:31:00-05:00,27.139999,27.215300,27.010000,27.139999,257534
2025-12-09 09:32:00-05:00,27.150000,27.232901,27.129999,27.139999,106428
2025-12-09 09:33:00-05:00,27.160000,27.180000,27.020000,27.150000,198462
2025-12-09 09:34:00-05:00,27.159000,27.170000,27.059999,27.160000,873079
...,...,...,...,...,...
2025-12-09 15:55:00-05:00,26.764999,26.770000,26.740000,26.759899,0
2025-12-09 15:56:00-05:00,26.770000,26.820000,26.770000,26.770000,299080
2025-12-09 15:57:00-05:00,26.838800,26.844999,26.760000,26.770000,347345


## Strategies

### Momentum long signal: rising EMA + high volume example

In [7]:
# Indicators
sofi_ema20_high_volume_df = sofi_df.copy()
sofi_ema20_high_volume_df["ema20"] = sofi_ema20_high_volume_df["Close"].ewm(span=20, adjust=False).mean()
sofi_ema20_high_volume_df["ema20_prev"] = sofi_ema20_high_volume_df["ema20"].shift(1)
sofi_ema20_high_volume_df["ema_rising"] = sofi_ema20_high_volume_df["ema20"] > sofi_ema20_high_volume_df["ema20_prev"]

sofi_ema20_high_volume_df["vol_ma20"] = sofi_ema20_high_volume_df["Volume"].rolling(20).mean()
sofi_ema20_high_volume_df["high_vol"] = sofi_ema20_high_volume_df["Volume"] > sofi_ema20_high_volume_df["vol_ma20"]

# Momentum long signal: price above rising EMA + high volume
sofi_ema20_high_volume_df["long_signal"] = (sofi_ema20_high_volume_df["Close"] > sofi_ema20_high_volume_df["ema20"]) & sofi_ema20_high_volume_df["ema_rising"] & sofi_ema20_high_volume_df["high_vol"]

# Backtest (very simplified, 1 position max, no fees/slippage)
capital = 10_000
position = 0
entry_price = 0.0
pnl = []

for i in range(1, len(sofi_ema20_high_volume_df)):
    t = sofi_ema20_high_volume_df.index[i]
    row = sofi_ema20_high_volume_df.iloc[i]
    prev_row = sofi_ema20_high_volume_df.iloc[i-1]

    # Flatten before close to avoid overnight
    if position != 0 and t.time() >= pd.to_datetime("15:55").time():
        pnl.append((row["Close"] - entry_price) * position)
        position = 0
        entry_price = 0.0
        continue

    # If flat and signal, enter
    if position == 0 and row["long_signal"]:
        entry_price = row["Close"]
        position = capital // entry_price  # all-in sizing
        continue

    # If in position, check TP / SL or exit conditions
    if position != 0:
        # take profit 1%
        if row["High"] >= entry_price * 1.01:
            exit_price = entry_price * 1.01
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0
            continue

        # stop loss 0.5%
        if row["Low"] <= entry_price * 0.995:
            exit_price = entry_price * 0.995
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0
            continue

        # momentum breakdown: close below EMA20
        if row["Close"] < row["ema20"]:
            exit_price = row["Close"]
            pnl.append((exit_price - entry_price) * position)
            position = 0
            entry_price = 0.0

total_pnl = sum(pnl)
print("Total PnL:", total_pnl)

sofi_ema20_high_volume_df


Total PnL: -107.57319164276123


Price,Close,High,Low,Open,Volume,ema20,ema20_prev,ema_rising,vol_ma20,high_vol,long_signal
Datetime,,,,,,,,,,,
2025-12-09 09:30:00-05:00,27.145000,27.610001,27.129999,27.610001,963090,27.145000,NaN,False,NaN,False,False
2025-12-09 09:31:00-05:00,27.139999,27.215300,27.010000,27.139999,257534,27.144524,27.145000,False,NaN,False,False
2025-12-09 09:32:00-05:00,27.150000,27.232901,27.129999,27.139999,106428,27.145046,27.144524,True,NaN,False,False
2025-12-09 09:33:00-05:00,27.160000,27.180000,27.020000,27.150000,198462,27.146470,27.145046,True,NaN,False,False
2025-12-09 09:34:00-05:00,27.159000,27.170000,27.059999,27.160000,873079,27.147663,27.146470,True,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2025-12-09 15:55:00-05:00,26.764999,26.770000,26.740000,26.759899,0,26.788632,26.791120,False,859604.30,False,False
2025-12-09 15:56:00-05:00,26.770000,26.820000,26.770000,26.770000,299080,26.786858,26.788632,False,867708.00,False,False
2025-12-09 15:57:00-05:00,26.838800,26.844999,26.760000,26.770000,347345,26.791805,26.786858,True,881670.30,False,False


### Golden Ratio Identifier

In [8]:
def find_swings(df, window=5):
    """Detect swing highs/lows using pivot points"""
    highs = df['High'].rolling(window*2+1, center=True).apply(lambda x: x.iloc[window] if x.iloc[window] == x.max() else np.nan)
    lows = df['Low'].rolling(window*2+1, center=True).apply(lambda x: x.iloc[window] if x.iloc[window] == x.min() else np.nan)
    return highs.dropna(), lows.dropna()

def calc_golden_pocket(df, prices, window=5):
    """Calculate latest Golden Pocket zone"""
    highs, lows = find_swings(df, window)
    
    # Get most recent swing high and low
    recent_high = highs.iloc[-1] if len(highs) > 0 else df['High'].max()
    recent_low = lows.iloc[-1] if len(lows) > 0 else df['Low'].min()
    
    fib_range = abs(recent_high - recent_low)
    
    # Golden Pocket: 61.8% to 65% retracement from high
    gp_upper = recent_high - (0.618 * fib_range)
    gp_lower = recent_high - (0.65 * fib_range)
    
    in_zone = False
    for price in prices:
        if gp_lower <= price <= gp_upper:
            in_zone = True
            break

    return {
        'swing_high': recent_high,
        'swing_low': recent_low,
        'gp_upper': gp_upper,
        'gp_lower': gp_lower,
        'in_zone': in_zone
    }

In [9]:
# Usage
result = calc_golden_pocket(sofi_ema20_high_volume_df, [27.68])
print(f"Golden Pocket: {result['gp_lower']:.2f} - {result['gp_upper']:.2f}")
print(f"Current price in zone: {result['in_zone']}")

Golden Pocket: 27.02 - 27.05
Current price in zone: False


### RSI

In [10]:
def calculate_rsi(delta:pd.Series, window:int=14, smooth:bool=False) -> float:
   # Delta is changes in price i.e. data.diff().
   global avg_gain, avg_loss
   rsi = 0
   gains = delta.where(delta > 0, 0)
   losses = -delta.where(delta < 0, 0)

   # Initial 14 days, and the formula uses a positive value for the average loss.
   if not smooth:
      avg_gain = gains.rolling(window=window).mean().iloc[-1]
      avg_loss = losses.rolling(window=window).mean().iloc[-1]
      
   # Smooth the results so that the RSI only nears 100 or zero in a strongly trending market.
   else: 
      curr_gain = 0
      curr_loss = 0
      if delta.iloc[-1] > 0:
         curr_gain = delta.iloc[-1]
      else: 
         curr_loss = -delta.iloc[-1]
      avg_gain = ((avg_gain * 13) + curr_gain) / window
      avg_loss = ((avg_loss * 13) + curr_loss) / window

   rs = avg_gain / avg_loss
   rsi = 100 - (100 / (1 + rs))

   return rsi

### MACD

In [11]:
"""
The signal line is a nine-period EMA of the MACD line.
MACD is best used with daily periods, where the traditional settings of 26/12/9 days are the default.
"""
def calculate_macd(data:pd.Series, short_window:int=12, long_window:int=26, signal_window:int=9):
   short_ema = data.ewm(span=short_window, adjust=False).mean()
   long_ema = data.ewm(span=long_window, adjust=False).mean()
   macd = short_ema - long_ema
   signal = macd.ewm(span=signal_window, adjust=False).mean()
   return macd, signal

### EMA

In [12]:
# Functions to calculate EMA20 and if EMA_Rising.
def calc_ema20(close_col: pd.Series) -> float:
    ema_20 = close_col.ewm(span=20, adjust=False).mean().iloc[-1]

    return ema_20

def is_ema_rising(curr_ema: float, prev_ema: float) -> bool:
    return curr_ema > prev_ema

### Volume

In [13]:
# Functions to calculate Vol_MA20 up to 20 and High_Vol.
def calc_vol_ma20(vol_col: pd.Series) -> float:
    return vol_col.mean()

def has_high_vol(curr_vol: float, vol_ma20: float) -> bool:
    return curr_vol > vol_ma20

### Simulated Live Testing with Strategies

In [14]:
start_time = pd.Timestamp(f"{day} 09:30", tz="America/New_York")

time_range = pd.date_range(
    start=f"{day} 09:30",
    end=f"{day} 15:59",
    freq="1min",
    tz="America/New_York"   # match your original tz if needed
)

OHLCV_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
bool_cols = ["EMA_Rising", "Buy_Signal"]
float_cols = ['Open', 'High', 'Low', 'Close', 'Volume', "Price_Change", "Price_Change_Pct" "EMA20", "RSI", "EMA9", "MACD"]
all_cols = ["Open", "High", "Low", "Close", "Volume", "Price_Change", "Price_Change_Pct", "EMA20", "EMA_Rising", "RSI", "EMA9", "MACD", "Buy_Signal"]

# Intialize empty dataframe with all columns
template_row = {
            "Open": 0.0,
            "High": 0.0,
            "Low": 0.0,
            "Close": 0.0,
            'Volume': 0.0,
            "Price_Change": 0.0, 
            "Price_Change_Pct": 0.0,
            "EMA20": 0.0, 
            "EMA_Rising": False,
            "RSI": 0.0,
            "EMA9": 0.0,
            "MACD": 0.0,
            "Buy_Signal": False,
        }
empty_df = pd.DataFrame(template_row, index=[start_time])

empty_df[bool_cols] = empty_df[bool_cols].astype("boolean")
OHLCHistory = empty_df.copy()
OHLCHistory, OHLCHistory.dtypes

(                           Open  High  Low  Close  Volume  Price_Change  \
 2025-12-09 09:30:00-05:00   0.0   0.0  0.0    0.0     0.0           0.0   
 
                            Price_Change_Pct  EMA20  EMA_Rising  RSI  EMA9  \
 2025-12-09 09:30:00-05:00               0.0    0.0       False  0.0   0.0   
 
                            MACD  Buy_Signal  
 2025-12-09 09:30:00-05:00   0.0       False  ,
 Open                float64
 High                float64
 Low                 float64
 Close               float64
 Volume              float64
 Price_Change        float64
 Price_Change_Pct    float64
 EMA20               float64
 EMA_Rising          boolean
 RSI                 float64
 EMA9                float64
 MACD                float64
 Buy_Signal          boolean
 dtype: object)

In [15]:
time_range[0], time_range[-1]

(Timestamp('2025-12-09 09:30:00-0500', tz='America/New_York'),
 Timestamp('2025-12-09 15:59:00-0500', tz='America/New_York'))

In [16]:
fig = go.Figure(data=[go.Candlestick(
    x=sofi_df.index,
    open=sofi_df['Open'],
    high=sofi_df['High'],
    low=sofi_df['Low'],
    close=sofi_df['Close']
)])

"""
# Volume bars (scaled to fit price chart)
fig.add_trace(go.Bar(
    x=sofi_df.index,
    y=sofi_df['Volume'] / 1e6,  # Scale to millions
    name="Volume (M)",
    yaxis="y2",
    marker_color='rgba(100, 149, 237, 0.5)',
    opacity=1
))
"""
fig.update_layout(
    #yaxis=dict(title="Price"),
    #yaxis2=dict(title="Volume (M)", overlaying="y", side="right"),
    xaxis_rangeslider_visible=False,
    width=1200,
    height=800
)

fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"]), dict(bounds=[16, 9.5], pattern="hour")])
fig.show()

In [17]:
"""
start = '2025-12-17 11:55:00'
end   = '2025-12-17 12:20:00'
OHLCHistory["MACD_Above_Signal"] = OHLCHistory["MACD"] > OHLCHistory["EMA9"] 
slice = OHLCHistory.loc[start:end]
print(OHLCHistory.tail(20)[["RSI", "EMA9", "MACD", "MACD_Above_Signal"]])
"""

'\nstart = \'2025-12-17 11:55:00\'\nend   = \'2025-12-17 12:20:00\'\nOHLCHistory["MACD_Above_Signal"] = OHLCHistory["MACD"] > OHLCHistory["EMA9"] \nslice = OHLCHistory.loc[start:end]\nprint(OHLCHistory.tail(20)[["RSI", "EMA9", "MACD", "MACD_Above_Signal"]])\n'

### New Strat: Buy when Close > EMA20 and EMA20 is Rising, Sell at 1% Profit or 0.5% Loss

In [18]:
# Starting off stats (e.g. captial, position, and profit & losses)
capital = 10_000
position = 0 # max one position
entry_price = 0.0
pnl = []

for i in range(0, len(time_range)): 
    # Current_price is open price for each 1 min interval
    curr_time = time_range[i]
    open_price = sofi_df.loc[curr_time, "Open"]
    
    # Buy if signal indicates to buy
    if position == 0 and OHLCHistory.loc[curr_time, "Buy_Signal"]:
        entry_price = open_price
        position = capital // entry_price  # all-in sizing
        print(f"--- Iteration i={i} Buying ---")
        print(f"Current time: {curr_time}")
        print(f"Buy Price: {entry_price} for {capital} // {entry_price} -> {position} shares")
        print("-" * 50)

        
    # Wait for close and update data (usually in 1 min intervals)
    # time.sleep(0.1)

    # Updated OHLC
    OHLCHistory.loc[curr_time, OHLCV_cols] = sofi_df.loc[curr_time, OHLCV_cols]

    OHLCHistory["Price_Change"] = OHLCHistory["Close"].diff()
    OHLCHistory["Price_Change_Pct"] = OHLCHistory["Close"].pct_change() * 100
    OHLCHistory["EMA20"] = OHLCHistory["Close"].ewm(span=20, adjust=False).mean()
    OHLCHistory["EMA_Rising"] = OHLCHistory["EMA20"] > OHLCHistory["EMA20"].shift(1)
    
    OHLCHistory[bool_cols] = OHLCHistory[bool_cols].fillna(False).astype("boolean")
    
    if len(OHLCHistory) == 14:
        OHLCHistory.loc[curr_time, "RSI"] = calculate_rsi(OHLCHistory["Price_Change"])
    elif len(OHLCHistory) > 14:
        OHLCHistory.loc[curr_time, "RSI"] = calculate_rsi(OHLCHistory["Price_Change"], smooth=True)

    macd_col, ema9_col = calculate_macd(OHLCHistory["Close"])
    OHLCHistory["EMA9"] = ema9_col
    OHLCHistory["MACD"] = macd_col

    curr_row = OHLCHistory.iloc[-1]

    if i > 1: 
        curr_macd = float(curr_row["MACD"])
        curr_ema9 = float(curr_row["EMA9"])

        prev_macd = float(OHLCHistory["MACD"].iloc[-2])
        prev_ema9 = float(OHLCHistory["EMA9"].iloc[-2])

        bull_cross = (curr_macd > curr_ema9) and (prev_macd <= prev_ema9)
        bear_cross = (curr_macd < curr_ema9) and (prev_macd >= prev_ema9)

    # Extend dataframe for next time if under last time frame of time range
    if i < len(time_range) - 1:
        new_time = time_range[i+1]

        OHLCHistory.loc[new_time] = template_row

        # Only signal to buy if position does not exists
        if position == 0 and i > 1:
            # Check data and make a decision to buy in next interval
            good_price = curr_row["Close"] > curr_row["EMA20"]

            OHLCHistory.loc[new_time, "Buy_Signal"] = (good_price and curr_row["EMA_Rising"] and bear_cross)
    # Check data and make a decision to sell or not
    if position != 0:
        close_price = curr_row["Close"]

        # Flatten before close to avoid overnight
        # TODO: Change to "t.time() >= pd.to_datetime("15:59").time():"
        if curr_time.time() >= pd.to_datetime("15:59").time():
            exit_price = close_price
            pnl.append((exit_price - entry_price) * position)
            capital += pnl[-1]
            print(f"--- Iteration i={i} Selling Before Overnight ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Profit/Loss: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue
        
        # Take profit 1%
        if OHLCHistory.loc[curr_time, "Close"] >= entry_price * 1.01:
            exit_price = entry_price * 1.01
            pnl.append((exit_price - entry_price) * position)
            capital += pnl[-1]
            print(f"--- Iteration i={i} Selling 1.01x Profit ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Profit: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue

        # Bull Cross (Peak High)
        if bull_cross:
            exit_price = close_price
            pnl.append((exit_price - entry_price) * position)
            capital += pnl[-1]
            print(f"--- Iteration i={i} Selling Bull Cross Profit ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Profit: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue

        # Stop loss 0.5%
        if OHLCHistory.loc[curr_time, "Close"] <= entry_price * 0.995:
            exit_price = entry_price * 0.995
            pnl.append((exit_price - entry_price) * position)
            capital += pnl[-1]
            print(f"--- Iteration i={i} Selling 0.995x Loss ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Loss: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue
        
total_pnl = sum(pnl)
print("Total PnL:", total_pnl)

--- Iteration i=36 Buying ---
Current time: 2025-12-09 10:06:00-05:00
Buy Price: 27.31999969482422 for 10000 // 27.31999969482422 -> 366.0 shares
--------------------------------------------------
--- Iteration i=44 Selling 0.995x Loss ---
Current time: 2025-12-09 10:14:00-05:00
Sell Price: 27.183399696350097
Loss: -49.995599441528476 from (27.183399696350097 - 27.31999969482422) * 366.0 shares
Running Total: -49.995599441528476
--------------------------------------------------
--- Iteration i=108 Buying ---
Current time: 2025-12-09 11:18:00-05:00
Buy Price: 27.3971004486084 for 9950.004400558471 // 27.3971004486084 -> 363.0 shares
--------------------------------------------------
--- Iteration i=129 Selling 0.995x Loss ---
Current time: 2025-12-09 11:39:00-05:00
Sell Price: 27.260114946365356
Loss: -49.72573731422455 from (27.260114946365356 - 27.3971004486084) * 363.0 shares
Running Total: -99.72133675575303
--------------------------------------------------
Total PnL: -99.72133675

In [19]:
curr_macd

0.05073679512691953

In [20]:
OHLCHistory["Volume_Change"] = OHLCHistory['Volume'].diff()
OHLCHistory["Volume_Change_Pct"] = OHLCHistory['Volume'].pct_change() * 100
OHLCHistory.head(5)

,Open,High,Low,Close,Volume,Price_Change,Price_Change_Pct,EMA20,EMA_Rising,RSI,EMA9,MACD,Buy_Signal,Volume_Change,Volume_Change_Pct
2025-12-09 09:30:00-05:00,27.610001,27.610001,27.129999,27.145000,963090.0,NaN,NaN,27.145000,False,0.0,0.000000,0.000000,False,NaN,NaN
2025-12-09 09:31:00-05:00,27.139999,27.215300,27.010000,27.139999,257534.0,-0.005001,-0.018424,27.144524,False,0.0,-0.000080,-0.000399,False,-705556.0,-73.259612
2025-12-09 09:32:00-05:00,27.139999,27.232901,27.129999,27.150000,106428.0,0.010000,0.036847,27.145046,True,0.0,-0.000046,0.000091,False,-151106.0,-58.674194
2025-12-09 09:33:00-05:00,27.150000,27.180000,27.020000,27.160000,198462.0,0.010000,0.036833,27.146470,True,0.0,0.000218,0.001271,False,92034.0,86.475364
2025-12-09 09:34:00-05:00,27.160000,27.170000,27.059999,27.159000,873079.0,-0.000999,-0.003680,27.147663,True,0.0,0.000595,0.002102,False,674617.0,339.922504


In [21]:
"""
# Starting off stats (e.g. captial, position, and profit & losses)
capital = 10_000
position = 0 # max one position
entry_price = 0.0
pnl = []

for i in range(0, len(time_range)): 
    # Current_price is open price for each 5 min interval
    curr_time = time_range[i]
    current_price = sofi_df.loc[curr_time, "Open"]
    
    # Buy if signal indicates to buy
    if position == 0 and OHLCHistory.loc[curr_time, "Buy_Signal"]:
        entry_price = current_price
        position = capital // entry_price  # all-in sizing
        print(f"--- Iteration i={i} Buying ---")
        print(f"Current time: {curr_time}")
        print(f"Buy Price: {entry_price} for {capital} // {entry_price} -> {position} shares")
        print("-" * 50)

        
    # Wait for close and update data (usually in 5 mins intervals)
    # time.sleep(0.1)

    # Updated OHLC
    OHLCHistory.loc[curr_time, OHLCV_cols] = sofi_df.loc[curr_time, OHLCV_cols]

    OHLCHistory["EMA20"] = OHLCHistory["Close"].ewm(span=20, adjust=False).mean()
    OHLCHistory["Vol_EMA20"] = OHLCHistory["Volume"].rolling(20).mean()
    OHLCHistory["Price_Change"] = OHLCHistory["Close"].diff()
    OHLCHistory["Price_Change_Pct"] = OHLCHistory["Close"].pct_change() * 100
    OHLCHistory["EMA_Rising"] = OHLCHistory["EMA20"] > OHLCHistory["EMA20"].shift(1)
    OHLCHistory["High_Vol"] = OHLCHistory["Vol_EMA20"] > OHLCHistory["Vol_EMA20"].shift(1)
    #OHLCHistory["Volume"] > OHLCHistory["Vol_EMA20"] * 1.1
    
    OHLCHistory[bool_cols] = OHLCHistory[bool_cols].fillna(False).astype("boolean")

    # Extend dataframe for next time if under last time frame of time range
    if i < len(time_range) - 1:
        new_time = time_range[i+1]

        OHLCHistory.loc[new_time] = template_row

        # Check data and make a decision to buy in next interval
        curr_row = OHLCHistory.loc[curr_time]
        good_price = curr_row["Close"] > curr_row["EMA20"]

        # Only signal to buy if position does not exists
        if position == 0:
            OHLCHistory.loc[new_time, "Buy_Signal"] = good_price and curr_row["EMA_Rising"] #and curr_row["High_Vol"]
        
    # Check data and make a decision to sell or not
    if position != 0:
        # Flatten before close to avoid overnight
        # TODO: Change to "t.time() >= pd.to_datetime("15:55").time():"
        if curr_time.time() >= pd.to_datetime("15:55").time():
            exit_price = OHLCHistory.loc[curr_time, "Close"]
            pnl.append((exit_price - entry_price) * position)
            print(f"--- Iteration i={i} Selling Before Overnight ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Profit/Loss: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue

        # take profit 1%
        if OHLCHistory.loc[curr_time, "Close"] >= entry_price * 1.01:
            exit_price = entry_price * 1.01
            pnl.append((exit_price - entry_price) * position)
            print(f"--- Iteration i={i} Selling 1.01x Profit ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Profit: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue
        # stop loss 0.5%
        if OHLCHistory.loc[curr_time, "Close"] <= entry_price * 0.995:
            exit_price = entry_price * 0.995
            pnl.append((exit_price - entry_price) * position)
            print(f"--- Iteration i={i} Selling 0.995x Lost ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Loss: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue
        # momentum breakdown: close below EMA20
        if OHLCHistory.loc[curr_time, "Close"] < OHLCHistory.loc[curr_time, "EMA20"]:
            exit_price = OHLCHistory.loc[curr_time, "Close"]
            pnl.append((exit_price - entry_price) * position)
            print(f"--- Iteration i={i} Selling Below EMA20 ---")
            print(f"Current time: {curr_time}")
            print(f"Sell Price: {exit_price}")
            print(f"Loss: {pnl[-1]} from ({exit_price} - {entry_price}) * {position} shares")
            print(f"Running Total: {sum(pnl)}")
            print("-" * 50)
            position = 0
            continue

total_pnl = sum(pnl)
print("Total PnL:", total_pnl)
"""

'\n# Starting off stats (e.g. captial, position, and profit & losses)\ncapital = 10_000\nposition = 0 # max one position\nentry_price = 0.0\npnl = []\n\nfor i in range(0, len(time_range)): \n    # Current_price is open price for each 5 min interval\n    curr_time = time_range[i]\n    current_price = sofi_df.loc[curr_time, "Open"]\n\n    # Buy if signal indicates to buy\n    if position == 0 and OHLCHistory.loc[curr_time, "Buy_Signal"]:\n        entry_price = current_price\n        position = capital // entry_price  # all-in sizing\n        print(f"--- Iteration i={i} Buying ---")\n        print(f"Current time: {curr_time}")\n        print(f"Buy Price: {entry_price} for {capital} // {entry_price} -> {position} shares")\n        print("-" * 50)\n\n\n    # Wait for close and update data (usually in 5 mins intervals)\n    # time.sleep(0.1)\n\n    # Updated OHLC\n    OHLCHistory.loc[curr_time, OHLCV_cols] = sofi_df.loc[curr_time, OHLCV_cols]\n\n    OHLCHistory["EMA20"] = OHLCHistory["Close"